In [8]:
import sys
sys.path.append("..")
import os
os.chdir("..")

In [14]:
os.chdir('git/neural-analysis')

In [15]:
# load model from cache
# sample
import shelve
from conf import Conf
from models_utils import get_key_per_model
import main
import models
import data_manager
import features_lib

main.nid = 72
data = data_manager.Loader5()(main.nid)
dataprop = data_manager.DataProp1(data)

m = models.AlloModel(n_bats=5, max_iter=25, fit_intercept=False)
d = shelve.open(Conf().CACHE_FOLDER + "models")
m.neuron_id = main.nid
k = get_key_per_model(m)
m = d[k]

In [7]:
sorted(dataprop.data.columns.drop(features_lib.get_label_name()))

['BAT_0_F_HD',
 'BAT_0_F_X',
 'BAT_0_F_Y',
 'BAT_1_F_A',
 'BAT_1_F_D',
 'BAT_1_F_X',
 'BAT_1_F_Y',
 'BAT_2_F_A',
 'BAT_2_F_D',
 'BAT_2_F_X',
 'BAT_2_F_Y',
 'BAT_3_F_A',
 'BAT_3_F_D',
 'BAT_3_F_X',
 'BAT_3_F_Y',
 'BAT_4_F_A',
 'BAT_4_F_D',
 'BAT_4_F_X',
 'BAT_4_F_Y']

In [9]:
m.gam_model.summary()

PoissonGAM                                                                                                
=============================================== ==========================================================
Distribution:                       PoissonDist Effective DoF:                                     29.4854
Link Function:                          LogLink Log Likelihood:                                 -15016.018
Number of Samples:                        44546 AIC:                                            30091.0069
                                                AICc:                                             30091.05
                                                UBRE:                                               2.5388
                                                Scale:                                                 1.0
                                                Pseudo R-Squared:                                   0.0703
Feature Function                  Lam

C:\Users\ITAYY~1.WIS\AppData\Local\Temp/ipykernel_7256/3181566157.py:1: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  m.gam_model.summary()


In [10]:
set(m.build_covariates_list()) & set(dataprop.data.columns)

{'BAT_0_F_HD',
 'BAT_0_F_X',
 'BAT_0_F_Y',
 'BAT_1_F_X',
 'BAT_1_F_Y',
 'BAT_2_F_X',
 'BAT_2_F_Y',
 'BAT_3_F_X',
 'BAT_3_F_Y',
 'BAT_4_F_X',
 'BAT_4_F_Y'}

In [48]:
covariate_list = set(m.build_covariates_list()) & set(dataprop.data.columns)
X = dataprop.data[covariate_list]
y = dataprop.data[features_lib.get_label_name()]
samples = m.gam_model.sample(X, y, quantity='coef', n_bootstraps=1, n_draws=1000)

In [54]:
import numpy as np
from copy import deepcopy
import time

m2 = deepcopy(m)
mean_sample = np.mean(samples, axis=0)

def get_loglikelihood_per_sample(sample):
    m2.gam_model.coef_ = sample
    ll = m2.gam_model.loglikelihood(X, y)
    return ll

t = time.time()
print(get_loglikelihood_per_sample(samples[0]))
t2 = time.time()
print(t2 - t)
print(get_loglikelihood_per_sample(samples[1]))
t3 = time.time()
print(t3 - t2)

pDIC = 2*get_loglikelihood_per_sample(mean_sample) - 2*np.mean(list(map(get_loglikelihood_per_sample, samples)))
dic = -2*get_loglikelihood_per_sample(mean_sample) + 2*pDIC
print(dic)

-91625.01889627487
1.30251145362854
-108701.25542037014
1.3174755573272705
206937.87316468483


In [3]:
import numpy as np
from copy import deepcopy
import time

m_ego = models.EgoModel(n_bats=5, max_iter=25, fit_intercept=True)
m_ego.neuron_id = main.nid

def get_loglikelihood_per_sample(sample):
    global m, X, y
    m2 = deepcopy(m)
    m2.gam_model.coef_ = sample
    ll = m2.gam_model.loglikelihood(X, y)
    return ll

def calc_dic(m_ego):
    global m, X, y
    k_ego = get_key_per_model(m_ego)
    m_ego = d[k_ego]
    covariate_list = set(m_ego.build_covariates_list()) & set(dataprop.data.columns)
    X = dataprop.data[covariate_list]
    y = dataprop.data[features_lib.get_label_name()]
    samples_ego = m_ego.gam_model.sample(X, y, quantity='coef', n_bootstraps=1, n_draws=10)

    m2 = deepcopy(m_ego)
    mean_sample = np.mean(samples_ego, axis=0)

    pDIC_ego = 2*get_loglikelihood_per_sample(mean_sample) - 2*np.mean(list(map(get_loglikelihood_per_sample, samples_ego)))
    dic_ego = -2*get_loglikelihood_per_sample(mean_sample) + 2*pDIC_ego
    print(dic_ego)
    return dic_ego



NameError: name 'models' is not defined

In [2]:
m = models.AlloModel(n_bats=5, max_iter=25, fit_intercept=True)
m.neuron_id = 1003
k = get_key_per_model(m)
m = d[k]
dic_allo = calc_dic(m)

NameError: name 'models' is not defined

In [ ]:
m = models.EgoModel(n_bats=5, max_iter=25, fit_intercept=True)
m.neuron_id = 1003
k = get_key_per_model(m)
m = d[k]
dic_ego = calc_dic(m)

828045.5187325929


In [74]:
get_loglikelihood_per_sample(mean_sample)

-2283063.1374242883

In [68]:
d = shelve.open(Conf().CACHE_FOLDER + "models")
for i in list(d.keys()):
    if i.startswith("Ego") and "72" in i:
        print(i)
print(k_ego)

EgoModel|['BAT_2_F_A', 'BAT_2_F_D']|72|0
EgoModel|['BAT_2_F_A', 'BAT_2_F_D']|72|10000
EgoModel|['BAT_1_F_A', 'BAT_1_F_D', 'BAT_2_F_A', 'BAT_2_F_D', 'BAT_3_F_A', 'BAT_3_F_D', 'BAT_4_F_A', 'BAT_4_F_D']|72|0
EgoModel|['BAT_1_F_A', 'BAT_1_F_D', 'BAT_2_F_A', 'BAT_2_F_D', 'BAT_3_F_A', 'BAT_3_F_D', 'BAT_4_F_A', 'BAT_4_F_D']|72|0
